In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as keras_backend
from data_util import MetaDataLoader
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
import random
import sys
import time

import numpy as np
import matplotlib.pyplot as plt
print('Python version: ', sys.version)
print('TensorFlow version: ', tf.__version__)
tf.keras.backend.set_floatx('float64')
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('GPU found at: {}'.format(device_name))

Python version:  3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
TensorFlow version:  2.15.0
GPU found at: /device:GPU:0


In [ ]:
%cd '/content/drive/MyDrive/Meta_learning_research/Notebooks/'
import os
input_data = './samples/'
model_path = './models/'
prediction_path = './predicts/'
log_path = './logs/'

# Create the folder if it does not exist
os.makedirs(input_data, exist_ok=True)
os.makedirs(model_path, exist_ok=True)
os.makedirs(prediction_path, exist_ok=True)

# Avaiable backbones for Unet architechture
# 'vgg16' 'vgg19' 'resnet18' 'resnet34' 'resnet50' 'resnet101' 'resnet152' 'inceptionv3'
# 'inceptionresnetv2' 'densenet121' 'densenet169' 'densenet201' 'seresnet18' 'seresnet34'
# 'seresnet50' 'seresnet101' 'seresnet152', and 'attentionUnet'
backend = 'unet' # ResNet50 is the best model in the TL study

# Added first Convo 8 to 3 channels layers to the random init model
name = 'maml-model-' + backend + '-' + str(np.random.randint(1000000))

logdir = log_path + name
if(os.path.isdir(logdir)):
  shutil.rmtree(logdir)
os.makedirs(logdir, exist_ok=True)

print('model location: '+ model_path+name+'.h5')

/content/drive/MyDrive/Meta_learning_research/Notebooks
model location: ./models/maml-model-unet-761117.h5


In [ ]:
%cd '/content/drive/MyDrive/Meta_learning_research/Notebooks/'
data_dir = './samples/'  # Replace with the path to your directory containing numpy files
locations_meta_training = ['Alexander', 'Rowancreek']
locations_meta_testing = ['Covington']
num_samples_per_location = 10  # Configure the number of samples per location
num_episodes = 10  # Number of episodes

data_loader = MetaDataLoader(data_dir, num_samples_per_location)

# Create multi episodes for meta-training
mate_train_episodes = data_loader.create_multi_episodes(num_episodes, locations_meta_training)
mate_test_episodes = data_loader.create_multi_episodes(num_episodes, locations_meta_testing)

/content/drive/MyDrive/Meta_learning_research/Notebooks


In [ ]:
len(mate_train_episodes) #[0]["support_set_data"].shape

10

In [ ]:
class UnetModel(tf.keras.Model):
    def __init__(self, num_classes):
        super(UnetModel, self).__init__()
        # Encoder: Downsampling
        self.conv1 = Conv2D(64, 3, activation='relu', padding='same')
        self.conv2 = Conv2D(64, 3, activation='relu', padding='same')
        self.pool1 = MaxPooling2D(pool_size=(2, 2))

        self.conv3 = Conv2D(128, 3, activation='relu', padding='same')
        self.conv4 = Conv2D(128, 3, activation='relu', padding='same')
        self.pool2 = MaxPooling2D(pool_size=(2, 2))

        # Bottleneck
        self.conv5 = Conv2D(256, 3, activation='relu', padding='same')
        self.conv6 = Conv2D(256, 3, activation='relu', padding='same')

        # Decoder: Upsampling
        self.upconv1 = Conv2DTranspose(128, 2, strides=(2, 2), padding='same')
        self.conv7 = Conv2D(128, 3, activation='relu', padding='same')
        self.conv8 = Conv2D(128, 3, activation='relu', padding='same')

        self.upconv2 = Conv2DTranspose(64, 2, strides=(2, 2), padding='same')
        self.conv9 = Conv2D(64, 3, activation='relu', padding='same')
        self.conv10 = Conv2D(64, 3, activation='relu', padding='same')

        # Output Layer
        self.conv11 = Conv2D(num_classes, 1, activation='softmax')

    def forward(self, inputs):
        # Encoder
        c1 = self.conv1(inputs)
        c2 = self.conv2(c1)
        p1 = self.pool1(c2)

        c3 = self.conv3(p1)
        c4 = self.conv4(c3)
        p2 = self.pool2(c4)

        # Bottleneck
        c5 = self.conv5(p2)
        c6 = self.conv6(c5)

        # Decoder
        u1 = self.upconv1(c6)
        u1 = tf.concat([u1, c4], axis=-1)  # Skip connection
        c7 = self.conv7(u1)
        c8 = self.conv8(c7)

        u2 = self.upconv2(c8)
        u2 = tf.concat([u2, c2], axis=-1)  # Skip connection
        c9 = self.conv9(u2)
        c10 = self.conv10(c9)

        outputs = self.conv11(c10)
        return outputs

In [ ]:
def loss_function(pred_y, y):
    return keras_backend.mean(keras.losses.mean_squared_error(y, pred_y))

def np_to_tensor(list_of_numpy_objs):
    return (tf.convert_to_tensor(obj) for obj in list_of_numpy_objs)


def compute_loss(model, x, y, loss_fn=loss_function):
    logits = model.forward(x)
    mse = loss_fn(y, logits)
    return mse, logits

def train_batch(x, y, model, optimizer):
    tensor_x, tensor_y = np_to_tensor((x, y))
    with tf.GradientTape() as tape:
        loss, _ = compute_loss(model, x, y)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

In [ ]:
def copy_model(model, x=None):
    '''Copy model weights to a new model.

    Args:
        model: model to be copied.
        x: An input example. This is used to run
            a forward pass in order to add the weights of the graph
            as variables.
    Returns:
        A copy of the model.
    '''

    copied_model = SineModel()

    # If we don't run this step the weights are not "initialized"
    # and the gradients will not be computed.
    copied_model.forward(tf.convert_to_tensor(x))

    copied_model.set_weights(model.get_weights())
    return copied_model


#Regular training

In [ ]:
def train_reg(epochs, x, y, dataset,mdl=None, lr=0.001, log_steps=100):
    if mdl is not None :
        model = mdl
    else:
        model = UnetModel()
    optimizer = keras.optimizers.Adam(learning_rate = lr)
    losses = []
    for epoch in range(epochs):
        if log_steps is not None:
            print("====== Epoch : " +str(epoch)+ " ====== ")

        total_loss = 0
        curr_loss = 0
        tmp = 0
        for i, sinusoid_generator in enumerate(dataset):
            # x, y = sinusoid_generator.batch()
            loss = train_batch(x, y, model, optimizer)
            total_loss += loss
            curr_loss = total_loss / (i + 1.0)

            tmp = i
            if log_steps is not None:
                if i % log_steps == 0 and i > 0:
                    print('Step {}: loss = {}'.format(i, curr_loss))
        losses.append(curr_loss)
    plt.plot(losses)
    plt.xlabel("Adaptation steps")
    plt.title("Mean Absolute Error Performance (Normal)")
    plt.ylabel("Loss")
    plt.show()
    return model, np.array(losses)

In [ ]:
nn = train_reg(5, train_ds)

# Reptile Training

In [ ]:
def train_reptile( epochs, dataset, mdl=None, lr_inner=0.001, lr_outer=0.01, batch_size=1, log_steps=100, k=1):
    #Step 1 : initialize model
    if mdl is not None :
        model = mdl
    else:
        model = SineModel()
    inner_optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=lr_inner)
    outer_optimizer = keras.optimizers.Adam(learning_rate=lr_outer)
    losses = []

    # Step 2 : iteration
    for epoch in range(epochs):
        if log_steps is not None:
            print("====== Epoch : " +str(epoch)+ " ====== ")
        total_loss = 0

        start = time.time()
        #Step 3 & 4 : get sample task from dataset
        for i, t in enumerate(random.sample(dataset, len(dataset))):
            x, y = np_to_tensor(t.batch())
            model.forward(x)

            # save current parameter
            old_weights = model.get_weights()

            model_copy = copy_model(model, x)
            # Step 5 : Compute W with SGD
            for _ in range(k):
                loss = train_batch(x, y, model_copy, inner_optimizer)

            # Step 6 : update model parameter
            after_weights = model_copy.get_weights()
            step_size = lr_inner * (1 - epoch / epochs) # linear scheduling method
            new_weights = [ old_weights[i] + ((old_weights[i] - after_weights[i]) * step_size)
                           for i in range(len(model.weights))]
            model.set_weights(new_weights)

            # additional step for outer optimization
            if (i+1) % batch_size == 0:
                test_loss = train_batch(x, y, model, outer_optimizer)
            else:
                test_loss, logits = compute_loss(model, x, y)

            # Logs
            total_loss += test_loss
            loss = total_loss / (i+1.0)

            if log_steps is not None:
                if i % log_steps == 0 and i > 0:
                    print('Step {}: loss = {}, Time to run {} steps = {}'.format(i, loss, log_steps, time.time() - start))
                    start = time.time()

        losses.append(loss)
    plt.plot(losses)
    plt.xlabel("Adaptation steps")
    plt.title("Mean Absolute Error Performance (REPTILE)")
    plt.ylabel("Loss")
    plt.show()
    return model, np.array(losses)

In [ ]:
reptile = train_reptile(5, train_ds)

# Test

This test are comparing regular model vs reptile model with discrepancy in data compared to initial training. We would like to know how reptile perform while discrepancy getting bigger. The model will be trained in 100 task per discrepancy setup in 10 epochs each for 3 times and we will calculate the average of loss for better comparison variable.

In [ ]:
x_copy, _ = train_ds[0].batch()
def test_session(model, train_func):
    all_losses = []
    p = [0.3 , 0.5, 0.7, 0.9, 1]
    for i in range (0,5):
        title = "Amplitude : "+ str((i+2)) + " Phase : "+ str(p[i])+"*phi)"
        print(title)
        last_losses = 0
        all_res = []
        for j in range(0, 5):
            print("======= ", j+1, " run ========")
            test_task = [SinusoidGenerator(K=100, amplitude = i+2 , phase=p[i])]
            cp_model = copy_model(model, x_copy)
            res = train_func(epochs=100, mdl=cp_model, dataset=test_task, log_steps=None)
            all_res.append(res[1])
            last_losses+=res[1][-1]
        plt.plot(all_res[0])
        plt.plot(all_res[1])
        plt.plot(all_res[2])
        plt.plot(all_res[3])
        plt.plot(all_res[4])
        plt.title(title)
        plt.legend(["Run 1", "Run 2", "Run 3", "Run 4", "Run 5"], loc=(1.05, 0.5))
        plt.xlabel("Adaptation steps")
        plt.ylabel("Loss")
        plt.show()
        all_losses.append(last_losses/5)

    category = ["Test_Ds1", "Test_Ds2", "Test_Ds3","Test_Ds4","Test_Ds5"]
    fig, ax = plt.subplots()
    ax.plot(category, all_losses, label="loss")
    ax.legend(loc=(1.05, 0.5))
    plt.show()
    return all_losses

In [ ]:
reptile_res = test_session(reptile[0], train_reptile)